# Image of M4 + linked Gaia DR3 color-magnitude diagram

Download a DSS2 Red image of M4, and load it into Imviz:

In [ ]:
from jdaviz import Imviz
from astropy.coordinates import SkyCoord
from astroquery.skyview import SkyView
from astroquery.vizier import Vizier
import astropy.units as u
from jdaviz.configs.imviz.plugins.parsers import component_ids
from glue.core.component_link import ComponentLink
from glue.core.roi import RectangularROI

target_name = 'M4'

m4 = SkyCoord.from_name(target_name)
image = SkyView.get_images(target_name, survey='DSS2 Red', pixels=4000, radius=0.5 * u.deg)[0]

imviz = Imviz()
imviz.load_data(image, data_label=target_name)
imviz.show('sidecar:split-bottom')

Set up some plot options:

In [ ]:
plot_options = imviz.plugins['Plot Options']._obj
plot_options.axes_visible_value = True
plot_options.show_viewer_labels = True
plot_options.layer_selected = f'{target_name}[PRIMARY,1]'
plot_options.image_colormap_value = 'viridis'

Query the Gaia DR3 catalog for stars brighter than $V<10$  within 0.05$^\circ$ of M4:

In [ ]:
columns = ['source_id', 'ra', 'dec', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'parallax']
sources = Vizier(
    catalog='I/355/gaiadr3', columns=columns, row_limit=10_000
).query_region(
    m4, radius=0.05 * u.deg, column_filters={"Vmag": "<10"}
)[0]
sources['bp-rp'] = sources['BPmag'] - sources['RPmag']
sources.rename_columns(['RA_ICRS', 'DE_ICRS'], ['ra', 'dec'])

Remove units from these QTable columns:

In [ ]:
for col in sources.colnames:
    if hasattr(sources[col], 'quantity'):
        sources[col] = sources[col].quantity.value

imviz.load_data(sources)

In [ ]:
data1, data2 = imviz.app.data_collection[:2]

links = [
    ComponentLink([data1.id['Right Ascension']], data2.id['ra']),
    ComponentLink([data1.id['Declination']], data2.id['dec'])
]

for link in links:
    imviz.app.data_collection.add_link(link)

Set the scatter plot axes to be the BP-RP color and BP magnitude, flip y-axis limits so brightest sources are at the top of the color magnitude diagram:

In [ ]:
imviz.app.add_data_to_viewer('imviz-1', 'catalog')
scatter_viewer = imviz.viewers['imviz-1']._obj

scatter_viewer.state.x_att = imviz.app.data_collection[1].id['bp-rp']
scatter_viewer.state.y_att = imviz.app.data_collection[1].id['BPmag']

scatter_viewer.state.y_lim_helper.flip_limits()
scatter_viewer.set_limits(x_min=0, x_max=2.5)

In [ ]:
plot_options.layer_selected = 'catalog'
plot_options.marker_color_value = 'gray'
plot_options.marker_opacity_value = 0.6
plot_options.marker_size_scale_value = 10

Select horizontal branch stars from the color-magnitude diagram:

In [ ]:
imviz.viewers['imviz-1']._obj.apply_roi(RectangularROI(0.5, 1.5, 10, 14))

Make the subset markers easier to see:

In [ ]:
plot_options.layer_selected = 'Subset 1'
plot_options.marker_color_value = 'r'
plot_options.marker_opacity_value = 0.9
plot_options.marker_size_scale_value = 10

In [ ]:
plot_options.layer_selected = 'Subset 2'
plot_options.marker_color_value = 'g'
plot_options.marker_opacity_value = 0.9
plot_options.marker_size_scale_value = 10